In [94]:
import pandas as pd
import numpy as np
from sklearn import model_selection
from sklearn import tree
from sklearn import preprocessing

def get_dataset_df(ruta, header):
    '''
    El csv con los datos debe tener en la ultima columna el atributo objetivo. El parámetro header debe ser True si el csv
    contiene header, false en caso contrario. Para que la ruta funcione correctamente, meter el csv en la carpeta docs y pasar
    la ruta de la siguiente manera: "docs/titanic.csv"
    Este metodo devuelve los datos del csv como dataframe.
    '''
    r = str(ruta)
    if header == False:
        dataset = pd.read_csv(r, header=None)
    if header == True:
        dataset = pd.read_csv(r)
    
    return dataset

In [95]:
def get_atributos(dataset):
    '''
    Recibe como parámetro el dataframe obtenido en el método get_dataset_df
    Devuelve las columnas de atributos del dataframe
    '''
    columnas = dataset.columns.tolist()
    atributos = dataset.iloc[:, 0:len(columnas)-1]

    return atributos

In [88]:
def metodo_evaluacion_robusta(dataset, atributos, N_EXP, CV):
    
    '''
    Devuelve la media de las tasas de aciertos del algoritmo obtenidas en cada iteracion
    '''
    
    columnas = dataset.columns.tolist() # Pasamos todos los nombres de las columnas del dataset a una lista
    nombre_objetivo = columnas.pop(len(columnas)-1) # Eliminamos de la lista anterior la ultima columna (que se corresponde
                                                # con la columna objetivo) y guardamos su nombre en la variable nombre_objetivo
    objetivo = dataset[nombre_objetivo] # Obtenemos la columna objetivo del dataset

    codificador_atributos = preprocessing.OrdinalEncoder() # Codificador adecuado para los atributos
    codificador_atributos.fit(atributos)  # Ajusta la codificación a los atributos
    atributos_codificados = codificador_atributos.transform(atributos)

    codificador_objetivo = preprocessing.LabelEncoder() # Codificador adecuado para el objetivo
    objetivo_codificado = codificador_objetivo.fit_transform(objetivo) # El método fit_transform ajusta la codificación y la aplica a los datos
    
    atributos_entrenamiento, atributos_prueba, objetivo_entrenamiento, objetivo_prueba = model_selection.train_test_split(
    atributos_codificados, objetivo_codificado,  # Conjuntos de datos a dividir, usando los mismos índices para ambos
    random_state=12345,  # Valor de la semilla aleatoria, para que el muestreo sea reproducible, a pesar de ser aleatorio
    test_size=.20  # Tamaño del conjunto de prueba
    )  

    clasif_arbol_decision = tree.DecisionTreeClassifier() # creamos el clasificador
    clasif_arbol_decision.fit(X=atributos_entrenamiento, y=objetivo_entrenamiento) 
    
    lista_promedios = [] # Inicializamos una lista, en la que iremos metiendo los promedios

    for i in range(N_EXP):
        scores = model_selection.cross_val_score(estimator=clasif_arbol_decision, X=atributos_prueba, y=objetivo_prueba, cv=CV, scoring='balanced_accuracy')
        # La variable scores almacena la tasa de aciertos del algoritmo sobre los datos que le pasamos como parámetros
        promedio = scores.mean() # Calculamos el promedio como la media los scores
        lista_promedios.append(promedio) #Añadimos el promedio a la lista de promedios
    
    media = sum(lista_promedios)/len(lista_promedios) # Calculamos la media de los promedios almacenados en la lista, que es 
                                                        # el dato que devolvera este método
    return media
    

In [141]:
def algoritmo_sfs(dataset, D=None):
    
    '''
    Devuelve las K combinaciones de variables que proporcionan mayor rendimiento, acompañadas de su rendimiento y tamaño.
    Todo esto mostrado en una tabla
    '''
    
    solucion_actual = []    # Inicializamos la lista en la que almacenaremos la solucion actual
    lista_auxiliar = []     # Inicializamos una lista auxiliar
    solucion = []           # Esta lista se mostrara en la tabla. Contendra las mejores combinaciones de variables
    lista_mejores_promedios = [] # Esta lista se mostrara en la tabla. Contendra los mejores promedios
    size = []               # Esta lista se mostrara en la tabla. Contendra los K tamaños de la lista solucion
    k=0                     # inicializamos K
    variables_predictoras = dataset.columns.tolist() # Metemos en variables_predictoras los nombres de todas las columnas del dataset
    nombre_objetivo = variables_predictoras.pop(len(variables_predictoras)-1) # Eliminamos de variables predictoras el nombre de la columna objetivo
                                                                    # Y almacenamos en nombre_objetivo dicho nombre
        
    if D is None:
        D = len(variables_predictoras)    
        
        
    objetivo = dataset[nombre_objetivo] # Obtenemos la columna objetivo del dataset
    variables_sin_añadir = variables_predictoras # en variables sin añadir almacenamos los nombres de todas las 
                                                # columnas del dataset, exceptuando la del objetivo, eliminada anteriormente
    i=1
    while k < D:
        lista_scores = [] #Inicializamos 3 listas mas, necesarias para que todos los datos se muestren en la tabla correctamente
        lista_sol = []
        solucion_auxiliar = []
        for v in variables_sin_añadir: # recorremos las variables que aun no han sido añadidas
            lista_sol = list(solucion_actual)
            lista_sol.append(dataset[v]) # Añadimos a lista_sol la columna del dataset cuyo nombre es la variable v
            solucion_temporal = list(lista_sol) # Pasamos a solucion_temporal la lista_sol
            solucion_temporal = np.reshape(np.ravel(lista_sol), (len(objetivo),1+k)) # Transformamos el tamaño de la solucion 
                                                                 # temporal para que la solucion temporal tenga el mismo numero
                                                                 # de filas que el objetivo
            score = metodo_evaluacion_robusta(dataset, solucion_temporal, 10, 10) # Calculamos el score
            lista_scores.append(score) # Añadimos el score a la lista de scores
            i=i+1 # Aumentamos i
        mejor_promedio = np.amax(lista_scores) # Calculamos el mejor score de la lista de scores
        mejor_solucion_temporal = variables_sin_añadir[lista_scores.index(mejor_promedio)] # La mejor solucion temporal sera la
                                                                 # variable que haya obtenido el mejor promedio
            
        solucion_actual.append(dataset[mejor_solucion_temporal]) # Añadimos a solucion actual la columna del dataset cuyo nombre
                                                                 # es la mejor solucion temporal
        lista_auxiliar.append(mejor_solucion_temporal) # Añadimos dicha variable a una lista auxiliar
        solucion_auxiliar = solucion_auxiliar + lista_auxiliar # Añadimos la lista auxiliar a otra lista para no tener problemas
                                                               # a la hora de mostrar la tabla
        solucion.append(solucion_auxiliar) # Añadimos a la lista solucion la lista solucion_auxiliar
        lista_mejores_promedios.append(mejor_promedio) # Añadimos a la lista de mejores promedios el mejor promedio
        size.append(len(solucion))   # Añadimos a la lista size el tamaño de la lista solucion
        variables_sin_añadir.remove(variables_sin_añadir[lista_scores.index(mejor_promedio)]) # Borramos de variables_sin_añadir
                                                                                              # la variable añadida
        k=k+1 # Aumentamos K

    print("\nTabla respuesta:\n")
    
    frame_data={'solution':solucion, 'score':lista_mejores_promedios, 'size':size}
    df=pd.DataFrame(frame_data)

    return df.sort_values(by=['score'], ascending=False)

In [147]:
def algoritmo_sffs(dataset):
    
    
    '''
    Devuelve las K combinaciones de variables obtenidas en cada iteración, acompañadas de su rendimiento y tamaño.
    Todo esto mostrado en una tabla
    '''
    
    solucion_actual = []
    añadidos = []
    eliminados = []
    k=0

    variables_sin_añadir = dataset.columns.tolist()
    variables_sin_eliminar = dataset.columns.tolist()
    variables = dataset.columns.tolist()                                           # En esta parte del código lo único que hacemos
    nombre_objetivo = variables_sin_añadir.pop(len(variables_sin_añadir)-1)       # es declarar e inicializar las listas que vamos
    nombre_objetivo2 = variables_sin_eliminar.pop(len(variables_sin_eliminar)-1)  # a usar, además de los índices i, j y K.
    nombre_objetivo3 = variables.pop(len(variables)-1)
    objetivo = dataset[nombre_objetivo]
    solucion = []
    
    lista_promedios = []
    solucion2 = []
    size = []
    i = 1
    j=0

    while not (len(añadidos) == len(variables) and k==10): # Mientras no se cumpla la condición de parada
        solucion_auxiliar = []
        lista_scores2 = [] # Inicializamos mas listas
        lista_sol2 = []
        if not len(añadidos) == len(variables): # si aun no se han añadido todas las variables
            lista_scores = []
            lista_sol = []
            for v in variables_sin_añadir: # recorremos las variables que aun no han sido añadidas
                lista_sol = list(solucion_actual)
                lista_sol.append(dataset[v]) # Añadimos a lista_sol la columna del dataset cuyo nombre es la variable v
                solucion_temporal = list(lista_sol) # Pasamos a solucion_temporal la lista_sol
                solucion_temporal = np.reshape(np.ravel(lista_sol), (len(objetivo),1+j)) # Transformamos el tamaño de la solucion
                                                                                        # temporal para que la solucion temporal
                                                                                        # tenga el mismo número de filas que el
                                                                                        # objetivo
                score = metodo_evaluacion_robusta(dataset, solucion_temporal, 10, 10) # Calculamos el score
                lista_scores.append(score) # Añadimos el score a la lista de scores
                i=i+1 # Aumentamos i
            mejor_promedio = np.amax(lista_scores) # Calculamos el mejor score de la lista de scores
            mejor_solucion_temporal = variables_sin_añadir[lista_scores.index(mejor_promedio)] # La mejor solucion temporal sera
                                                                            # la variable que haya obtenido el mejor promedio
            solucion_actual.append(dataset[mejor_solucion_temporal]) # Añadimos a solucion actual la columna del dataset cuyo
                                                                 # nombre es la mejor solucion temporal
            añadidos.append(mejor_solucion_temporal) 
            solucion.append(mejor_solucion_temporal) # Añadimos tanto a la lista añadidos como a la listasolucion la mejor 
                                                    #solucion temporal
            variables_sin_añadir.remove(variables_sin_añadir[lista_scores.index(mejor_promedio)]) # Eliminamos de variable sin
                                                                                                # añadir la variable añadida
        else:
            i=i-1 # Cuando se hayan añadido todas las variables, solo decrementaremos i

        if not len(solucion) == 1: # Si la longitud de la lista solucion no es 1

            for v in solucion: # Recorremos las variables añadidas a la lista solucion
                i=i-1 # Decrementamos la i
                if not variables_sin_eliminar.index(v) == 0: # Si la variable a eliminar no esta en la posicion 0
                    old_index = variables_sin_eliminar.index(v) 
                    variables_sin_eliminar.insert(0, variables_sin_eliminar.pop(old_index)) # Movemos dicha variable a la posicion 0
                lista_sol2 = list(solucion_actual)
                lista_sol2.remove(lista_sol2[variables_sin_eliminar.index(v)]) # Eliminamos de lista_sol2 la columna correspondiente
                                                                                # a dicha variable
                solucion_temporal = list(lista_sol2) # Igualamos solucion temporal a lista sol 2
                solucion_temporal = np.reshape(np.ravel(lista_sol2), (len(objetivo), j)) # Transformamos el tamaño de la lista
                                                                                        # sol 2 para que tenga el mismo número
                                                                                        # de filas que el objetivo
                score = metodo_evaluacion_robusta(dataset, solucion_temporal, 10, 10) # Calculamos el score
                lista_scores2.append(score) # Metemos el score en la lista de scores 2
            mejor_promedio2 = np.amax(lista_scores2) # Calculamos el mejor promedio 2
            mejor_solucion_temporal2 = variables_sin_eliminar[lista_scores2.index(mejor_promedio2)] #La mejor solucion temporal 2
                                                                        # sera la variable que haya obtenido el mejor promedio 2
            if mejor_promedio2 > mejor_promedio: # Si el mejor promedio 2 es mayor que el mejor promedio
                lista_promedios.append(mejor_promedio2) # Añadimos el mejor promedio 2 a la lista de promedios
                if not solucion.index(mejor_solucion_temporal2) == 0: # Si el indice de la mejor solucion temporal 2 en la lista
                                                                      # solucion no es 0
                    old_index = solucion.index(mejor_solucion_temporal2)
                    solucion.insert(0, solucion.pop(old_index))
                    solucion_actual.insert(0, solucion_actual.pop(old_index)) # Movemos dicha variable a la posicion 0, tanto
                                                                            # en la lista solucion, como en la solucion_actual
                solucion_actual.remove(solucion_actual[solucion.index(mejor_solucion_temporal2)]) # Eliminamos de la solucion
                                                                            # actual la columna correspondiente a la mejor solucion 
                                                                                # temporal 2
                variables_sin_eliminar.remove(mejor_solucion_temporal2) # Eliminamos de la lista de variables sin eliminar la
                                                                            # mejor solucion temporal 2
                eliminados.append(mejor_solucion_temporal2) # Añadimos a la lista de eliminados la mejor solucion temporal 2
                solucion.remove(mejor_solucion_temporal2) # Eliminamos de la lista solucion la mejor solucion temporal 2
                if len(añadidos) == len(variables): # Si la lista añadidos contiene todas las variables
                    mejor_promedio = mejor_promedio2 
                i=i-1
                j=j-1
                k=0 # Decrementamos i y j, y ponemos K a 0
            else:
                lista_promedios.append(mejor_promedio) # Si el mejor promedio 2 no es mayor que el mejor promedio, añadimos este
                                                        # a la lista de promedios
        else:
            lista_promedios.append(mejor_promedio) # Si la longitud de la lista solucion es 1, añadimos el mejor promedio a
                                                    # la lista de promedios
                
        if not len(añadidos) == len(variables): # Si añadidos no contiene todas las variables
            j=j+1 # Aumentamos j
        k=k+1 # Aumentamos K
        
        solucion_auxiliar = solucion_auxiliar + solucion # Añadimos la lista solucion a una lista auxiliar para no tener
                                                        # problemas a la hora de mostrar la tabla
        solucion2.append(solucion_auxiliar) # Metemos solucion auxiliar en la lista solucion2
        size.append(len(solucion)) # Añadimos el tamaño de la lista solucion en cada iteracion
        
        
    print("\nTabla respuesta:\n")
    
    frame_data={'solution':solucion2, 'score':lista_promedios, 'size':size}
    df=pd.DataFrame(frame_data)

    return df.sort_values(by=['score'], ascending=False)

In [152]:
dataset = get_dataset_df('docs/breastCancerDataset.csv', True)
atributos = get_atributos(dataset)
algoritmo_sffs(dataset)


Tabla respuesta:



,solution,score,size
0,[mean concave points],0.865893,1
1,[mean concave points],0.859286,1
13,"[mean concave points, symmetry error, mean tex...",0.664964,9
8,"[mean concave points, symmetry error, concave ...",0.661161,6
3,"[mean concave points, concavity error, smoothn...",0.658482,3
7,"[mean concave points, smoothness error, symmet...",0.648089,6
36,"[symmetry error, fractal dimension error, area...",0.641714,14
35,"[symmetry error, fractal dimension error, area...",0.641714,14
34,"[symmetry error, fractal dimension error, area...",0.641714,14
33,"[symmetry error, fractal dimension error, area...",0.641714,14
